# 궁동 맛집 정리하기

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
url_base = "https://www.diningcode.com"
url_sub = "/list.php?query=%EB%8C%80%EC%A0%84%EA%B6%81%EB%8F%99"
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0" name="viewport">
<meta content="kr" name="content-language"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="'대전궁동'에 대한 맛집 검색 결과는 모두 329곳 입니다. '대전궁동'의 상위권 추천맛집들에 대한 맛집블로그,사진,특색,지도,위치,전화번호 등의 정보를 확인하세요." name="description"/>
<meta content="'대전궁동' 맛집 추천순위 | 빅데이터 맛집검색, 다이닝코드" property="og:title">
<meta content="https://www.diningcode.com/list.php?query=%EB%8C%80%EC%A0%84%EA%B6%81%EB%8F%99" property="og:url">
<meta content="다이닝코드" property="og:site_name">
<meta content="'대전궁동'에 대한 맛집 검색 결과는 모두 329곳 입니다. '대전궁동'의 상위권 추천맛집들에 대한 맛집블로그,사진,특색,지도,위치,전화번호 등의 정보를 확인하세요." property="og:description">
<meta content="https://s3-ap-northeast-1.amazonaws.com/dcreviewsresized/300_300_20200302104528_photo1_e4885f69084c.jpg" property="og:image

In [3]:
rest_list = soup.find_all(class_='blink')
rest_list[0].parent.find('a')['href']

'/profile.php?rid=6Grb73j0bZTF'

In [4]:
tmp_one = rest_list[0]
tmp_one

<a class="blink" href="/profile.php?rid=6Grb73j0bZTF" target="_blank">
<span class="img" style="overflow:hidden; background:url('https://s3-ap-northeast-1.amazonaws.com/dcreviewsresized/300_300_20200302104528_photo1_e4885f69084c.jpg') no-repeat; background-size:auto 100px; background-position:center;"></span>
<span class="btxt">1. 오늘은파스타</span>
<span class="stxt">파스타, 필라프</span>
<span class="ctxt">데이트하기좋은, 분위기좋은, 데이트</span>
<span class="ctxt"><i class="loca">궁동</i>대전광역시 유성구 궁동 412-14</span>
<span class="icons">
</span>
</a>

In [5]:
tmp_one.find(class_='btxt')

<span class="btxt">1. 오늘은파스타</span>

In [6]:
tmp_one.find(class_='btxt').get_text()

'1. 오늘은파스타'

In [7]:
tmp_one.find(class_='btxt').get_text().split('.')

['1', ' 오늘은파스타']

In [8]:
cafe = tmp_one.find(class_='btxt').get_text().split('.')[1].replace(' ','')
cafe

'오늘은파스타'

In [9]:
menu = tmp_one.find(class_='stxt').get_text()
menu

'파스타, 필라프'

In [10]:
import re
import pandas as pd
from urllib.parse import urljoin

In [11]:
cafe_name = []
main_menu = []
url_add = []

for rest in rest_list:
    cafe_name.append(rest.find(class_='btxt').get_text().split('.')[1].replace(' ',''))
    main_menu.append(rest.find(class_='stxt').get_text())
    url_add.append(urljoin(url_base, rest.parent.find('a')['href']))

In [12]:
df = pd.DataFrame({'cafe':cafe_name,'menu':main_menu,'URL':url_add})
df

,cafe,menu,URL
0,오늘은파스타,"파스타, 필라프",https://www.diningcode.com/profile.php?rid=6Gr...
1,대전통닭본점,"바베큐, 치킨",https://www.diningcode.com/profile.php?rid=1OF...
2,이비가짬뽕,"굴짬뽕, 짬뽕",https://www.diningcode.com/profile.php?rid=xrw...
3,누오보나폴리,"화덕피자, 피자",https://www.diningcode.com/profile.php?rid=VAB...
4,천복순대국밥,"순대국밥, 국밥",https://www.diningcode.com/profile.php?rid=quB...
5,소신,"디저트카페, 카페",https://www.diningcode.com/profile.php?rid=niK...
6,동해원,"짬뽕, 짬뽕전문점",https://www.diningcode.com/profile.php?rid=18p...
7,정통집충남대점,돼지김치구이,https://www.diningcode.com/profile.php?rid=Jtg...
8,유소춘의낙곱새,"낙곱새, 꼬막비빔밥",https://www.diningcode.com/profile.php?rid=S6t...
9,악어식탁,"함박스테이크, 크림파스타",https://www.diningcode.com/profile.php?rid=1cj...


In [13]:
df.to_csv('gungdong1.csv',sep=',',encoding="utf-8")

In [14]:
df['URL'][0]

'https://www.diningcode.com/profile.php?rid=6Grb73j0bZTF'

In [15]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')

In [16]:
tmp_addr = soup_tmp.find(class_='locat').get_text()
tmp_addr

'대전광역시 유성구 궁동 412-14'

In [17]:
tmp_tel = soup_tmp.find(class_='tel').get_text()
tmp_tel

'042-826-0198'

In [18]:
from tqdm.notebook import tqdm
import time

In [19]:
addr = []
tel = []
for n in tqdm(df.index):
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html,'html.parser')
    
    addr.append(soup_tmp.find(class_='locat').get_text())
    tel.append(soup_tmp.find(class_='tel').get_text())

In [20]:
addr

['대전광역시 유성구 궁동 412-14',
 '대전광역시 유성구 궁동 471-11',
 '대전시 유성구 궁동 462-12',
 '대전시 유성구 궁동 406-10 3층',
 '대전광역시 유성구 궁동 471-8',
 '대전광역시 유성구 궁동 429-5',
 '대전시 유성구 궁동 414-5',
 '대전광역시 유성구 궁동 410-8',
 '대전광역시 유성구 궁동 411-9',
 '대전광역시 유성구 궁동 397-9']

In [21]:
tel

['042-826-0198',
 '042-825-2292',
 '042-823-7484',
 '042-322-9582',
 '042-822-1096',
 '010-8558-9746',
 '042-823-3495',
 '042-823-9289',
 '042-823-9933',
 '042-825-5201']

In [22]:
df2 = df.iloc[:,:-1]
df2

,cafe,menu
0,오늘은파스타,"파스타, 필라프"
1,대전통닭본점,"바베큐, 치킨"
2,이비가짬뽕,"굴짬뽕, 짬뽕"
3,누오보나폴리,"화덕피자, 피자"
4,천복순대국밥,"순대국밥, 국밥"
5,소신,"디저트카페, 카페"
6,동해원,"짬뽕, 짬뽕전문점"
7,정통집충남대점,돼지김치구이
8,유소춘의낙곱새,"낙곱새, 꼬막비빔밥"
9,악어식탁,"함박스테이크, 크림파스타"


In [23]:
df2['addr'] = addr
df2['tel'] = tel
df2

,cafe,menu,addr,tel
0,오늘은파스타,"파스타, 필라프",대전광역시 유성구 궁동 412-14,042-826-0198
1,대전통닭본점,"바베큐, 치킨",대전광역시 유성구 궁동 471-11,042-825-2292
2,이비가짬뽕,"굴짬뽕, 짬뽕",대전시 유성구 궁동 462-12,042-823-7484
3,누오보나폴리,"화덕피자, 피자",대전시 유성구 궁동 406-10 3층,042-322-9582
4,천복순대국밥,"순대국밥, 국밥",대전광역시 유성구 궁동 471-8,042-822-1096
5,소신,"디저트카페, 카페",대전광역시 유성구 궁동 429-5,010-8558-9746
6,동해원,"짬뽕, 짬뽕전문점",대전시 유성구 궁동 414-5,042-823-3495
7,정통집충남대점,돼지김치구이,대전광역시 유성구 궁동 410-8,042-823-9289
8,유소춘의낙곱새,"낙곱새, 꼬막비빔밥",대전광역시 유성구 궁동 411-9,042-823-9933
9,악어식탁,"함박스테이크, 크림파스타",대전광역시 유성구 궁동 397-9,042-825-5201


In [24]:
df2.set_index('cafe',inplace=True)
df2

,menu,addr,tel
cafe,,,
오늘은파스타,"파스타, 필라프",대전광역시 유성구 궁동 412-14,042-826-0198
대전통닭본점,"바베큐, 치킨",대전광역시 유성구 궁동 471-11,042-825-2292
이비가짬뽕,"굴짬뽕, 짬뽕",대전시 유성구 궁동 462-12,042-823-7484
누오보나폴리,"화덕피자, 피자",대전시 유성구 궁동 406-10 3층,042-322-9582
천복순대국밥,"순대국밥, 국밥",대전광역시 유성구 궁동 471-8,042-822-1096
소신,"디저트카페, 카페",대전광역시 유성구 궁동 429-5,010-8558-9746
동해원,"짬뽕, 짬뽕전문점",대전시 유성구 궁동 414-5,042-823-3495
정통집충남대점,돼지김치구이,대전광역시 유성구 궁동 410-8,042-823-9289
유소춘의낙곱새,"낙곱새, 꼬막비빔밥",대전광역시 유성구 궁동 411-9,042-823-9933


In [25]:
df2.to_csv('gungdong2.csv',sep=',',encoding="utf-8")

In [26]:
df3 = pd.read_csv('gungdong2.csv')

In [27]:
import numpy as np
import folium
import googlemaps

In [28]:
key_fd = open('googlemapskey.txt', mode='r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [29]:
gmaps = googlemaps.Client(key=gmaps_key)

In [30]:
lat = []
lng = []

for n in tqdm(df.index):
    if df3['addr'][n] != 'Multipl':
        target_name = df3['addr'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

In [31]:
df3['lat'] = lat
df3['lng'] = lng
df3.head()

,cafe,menu,addr,tel,lat,lng
0,오늘은파스타,"파스타, 필라프",대전광역시 유성구 궁동 412-14,042-826-0198,36.362308,127.348995
1,대전통닭본점,"바베큐, 치킨",대전광역시 유성구 궁동 471-11,042-825-2292,36.359897,127.342000
2,이비가짬뽕,"굴짬뽕, 짬뽕",대전시 유성구 궁동 462-12,042-823-7484,36.360435,127.340568
3,누오보나폴리,"화덕피자, 피자",대전시 유성구 궁동 406-10 3층,042-322-9582,36.361990,127.352960
4,천복순대국밥,"순대국밥, 국밥",대전광역시 유성구 궁동 471-8,042-822-1096,36.359831,127.342378


In [32]:
mapping = folium.Map(location=[df3['lat'].mean(), df3['lng'].mean()],zoom_start=11)
folium.Marker([df3['lat'].mean(), df3['lng'].mean()],popup='center').add_to(mapping)
mapping

In [33]:
mapping = folium.Map(location=[df3['lat'].mean(), df3['lng'].mean()],zoom_start=11)

for n in df3.index:
    if df3['addr'][n] != 'Multipl':
        folium.Marker([df3['lat'][n], df3['lng'][n]],popup=df3['cafe'][n]).add_to(mapping)
mapping